In [1]:
import ray
from ray import tune
import time

In [4]:
f=open('../head_node_info','r+')
buf=f.readlines()
buf=buf[0].strip().split()
address=buf[0]
redis_pass=buf[1]
f.close()

In [5]:
ray.init(address=address,_redis_password=redis_pass)

2023-02-13 00:13:29,355	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 10.109.75.157:45021...
2023-02-13 00:13:29,395	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at 10.109.75.157:33694 


Python version:,3.9.16
Ray version:,2.2.0
Dashboard:,http://10.109.75.157:33694


In [6]:
def objective(x, a, b):
    return a * (x ** 0.5) + b

In [7]:
def trainable (config):
    a,b = config['a'],config['b']
    # call the evalution funtion on diferent values of x
    for x in range(20):
        score = objective (x, a, b)
        # Send results to Tune
        tune.report(score = score)  

analysis = tune.run(trainable,
                    num_samples=100,
                    resources_per_trial={'cpu':1},
                    max_concurrent_trials=6,
                    config={
                        "a": tune.uniform(0, 20),
                        "b": tune.uniform(0,20)} )

In [8]:
analysis = tune.run(
    trainable,
    num_samples=20,
    stop={"training_iteration": 20},
    resources_per_trial={"cpu":1},
    config={"a": tune.uniform(0, 20),
            "b": tune.uniform(0,20)},
    max_concurrent_trials=8 )

Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,score,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
trainable_1b349_00000,2023-02-13_00-13-37,True,,94840e31b7634e229ddc84afaeca5f97,cn605-25-r,20,10.109.75.146,140611,51.3124,0.178755,0.00910783,0.178755,1676236417,0,,20,1b349_00000,0.0120587
trainable_1b349_00001,2023-02-13_00-13-45,True,,5892e1daa6754d6ca0d899affdfff2d4,cn605-27-l,20,10.109.75.157,264946,69.6087,4.7502,4.41209,4.7502,1676236425,0,,20,1b349_00001,0.00718784
trainable_1b349_00002,2023-02-13_00-13-41,True,,66762620ba7e477d98e4299dd43cdc28,cn605-27-l,20,10.109.75.157,264947,81.2848,0.308232,0.00657654,0.308232,1676236421,0,,20,1b349_00002,0.00832653
trainable_1b349_00003,2023-02-13_00-13-45,True,,09a841384f814ca1a3d15496afca7e3d,cn605-25-r,20,10.109.75.146,140661,18.8175,5.23731,0.0254576,5.23731,1676236425,0,,20,1b349_00003,0.0043838
trainable_1b349_00004,2023-02-13_00-13-41,True,,a97e673ed3bd4defb0a4851864918425,cn605-25-r,20,10.109.75.146,140662,13.8157,0.758024,0.0268388,0.758024,1676236421,0,,20,1b349_00004,0.00542808
trainable_1b349_00005,2023-02-13_00-13-45,True,,e84cd38eda3e4999a35394d71435cb61,cn605-25-r,20,10.109.75.146,140664,49.672,5.20038,4.42261,5.20038,1676236425,0,,20,1b349_00005,0.00784326
trainable_1b349_00006,2023-02-13_00-13-38,True,,94840e31b7634e229ddc84afaeca5f97,cn605-25-r,20,10.109.75.146,140611,88.3502,0.0884604,0.003057,0.0884604,1676236418,0,,20,1b349_00006,0.0120587
trainable_1b349_00007,2023-02-13_00-13-38,True,,94840e31b7634e229ddc84afaeca5f97,cn605-25-r,20,10.109.75.146,140611,36.1093,0.0968163,0.00245261,0.0968163,1676236418,0,,20,1b349_00007,0.0120587
trainable_1b349_00008,2023-02-13_00-13-38,True,,94840e31b7634e229ddc84afaeca5f97,cn605-25-r,20,10.109.75.146,140611,24.1918,0.0700519,0.00258875,0.0700519,1676236418,0,,20,1b349_00008,0.0120587
trainable_1b349_00009,2023-02-13_00-13-38,True,,94840e31b7634e229ddc84afaeca5f97,cn605-25-r,20,10.109.75.146,140611,24.4205,0.0910935,0.00201607,0.0910935,1676236418,0,,20,1b349_00009,0.0120587


2023-02-13 00:13:47,567	INFO tune.py:762 -- Total run time: 14.35 seconds (13.21 seconds for the tuning loop).


In [9]:
print("Best config is:", analysis.get_best_config(metric="score", mode="max"))

Best config is: {'a': 18.30500710783623, 'b': 16.214732789759452}


In [16]:
analysis.get_best_trial(metric='score',mode='max')

trainable_1b349_00018